# Vectara

>[Vectara](https://vectara.com/)是一种用于构建基于LLM的应用程序的API平台。它提供了一个简单易用的API，用于文档索引和查询，由Vectara管理，并针对性能和准确性进行了优化。


这个笔记本展示了如何使用与`Vectara`向量数据库相关的功能。

请参阅[Vectara API文档](https://docs.vectara.com/docs/)以获取更多有关如何使用API的信息。

我们想使用`OpenAIEmbeddings`，所以我们必须获取OpenAI API密钥。

In [1]:
import os
import getpass

os.environ['OPENAI_API_KEY'] = getpass.getpass('OpenAI API Key:')

OpenAI API Key:········


In [2]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Vectara
from langchain.document_loaders import TextLoader

In [3]:
loader = TextLoader('../../../state_of_the_union.txt')
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()

## 从LangChain连接到Vectara

Vectara API提供了简单的API端点，用于索引和查询。

In [4]:
vectara = Vectara.from_documents(docs, embedding=None)

## 相似性搜索

使用Vectara的最简单场景是执行相似性搜索。

In [5]:
query = "What did the president say about Ketanji Brown Jackson"
found_docs = vectara.similarity_search(query, n_sentence_context=0)

In [6]:
print(found_docs[0].page_content)

Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. 

Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. 

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.


## 带有分数的相似性搜索

有时我们可能想进行搜索，但也要获得相关性评分，以了解特定结果的好坏。

In [7]:
query = "What did the president say about Ketanji Brown Jackson"
found_docs = vectara.similarity_search_with_score(query)

In [8]:
document, score = found_docs[0]
print(document.page_content)
print(f"\nScore: {score}")

Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. 

Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. 

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.

Score: 0.7129974


## Vectara作为检索器

Vectara和所有其他向量存储一样，是一个LangChain检索器，使用余弦相似度。

In [9]:
retriever = vectara.as_retriever()
retriever

VectaraRetriever(vectorstore=<langchain.vectorstores.vectara.Vectara object at 0x122db2830>, search_type='similarity', search_kwargs={'lambda_val': 0.025, 'k': 5, 'filter': '', 'n_sentence_context': '0'})

In [10]:
query = "What did the president say about Ketanji Brown Jackson"
retriever.get_relevant_documents(query)[0]

Document(page_content='Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. \n\nTonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. \n\nOne of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. \n\nAnd I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.', metadata={'source': '../../../state_of_the_union.txt'})